# Training models for numerical data

This workbook illustrates how we can train a machine learning model for numerical data, with the focus on the visualization of the model. 

We train a CART decision tree model since we can explore how the model recognized the pattern in the data and how it could re-create it. 

In [ ]:
# read the file with data using openpyxl
import pandas as pd

# we read the data from the excel file, 
# which is the defect data from the ant 1.3 system
dfDataAnt13 = pd.read_excel('./chapter_6_dataset_numerical.xlsx', 
                            sheet_name='ant_1_3',
                            index_col=0)

In [ ]:
dfDataAnt13.head()

In [ ]:
# prepare the dataset
import sklearn.model_selection

X = dfDataAnt13.drop(['Defect'], axis=1)
y = dfDataAnt13.Defect

X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=42, train_size=0.9)

In [ ]:
# now that we have the data prepared
# we import the decision tree classifier and train it
from sklearn.tree import DecisionTreeClassifier

# first we create an empty classifier
decisionTreeModel = DecisionTreeClassifier()

# then we train the classifier
decisionTreeModel.fit(X_train, y_train)

# and we test it for the test set
y_pred_cart = decisionTreeModel.predict(X_test)

In [ ]:
# now, let's evaluate the code
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print(f'Accuracy: {accuracy_score(y_test, y_pred_cart):.2f}')
print(f'Precision: {precision_score(y_test, y_pred_cart, average="weighted"):.2f}, Recall: {recall_score(y_test, y_pred_cart, average="weighted"):.2f}')

In [ ]:
from sklearn.tree import export_text

tree_rules = export_text(decisionTreeModel, feature_names=list(X_train.columns))

print(tree_rules)

# Counter example

In this counter example, we use data from the same dataset, but for a different module. Let's see how the performance of the model differs based on the data. 

In [ ]:
# read the file with data using openpyxl
import pandas as pd

# we read the data from the excel file, 
# which is the defect data from the ant 1.3 system
dfDataCamel12 = pd.read_excel('./chapter_6_dataset_numerical.xlsx', 
                            sheet_name='camel_1_2',
                            index_col=0)

# prepare the dataset
import sklearn.model_selection

X = dfDataCamel12.drop(['Defect'], axis=1)
y = dfDataCamel12.Defect

X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=42, train_size=0.9)

In [ ]:
# now that we have the data prepared
# we import the decision tree classifier and train it
from sklearn.tree import DecisionTreeClassifier

# first we create an empty classifier
decisionTreeModelCamel = DecisionTreeClassifier()

# then we train the classifier
decisionTreeModelCamel.fit(X_train, y_train)

# and we test it for the test set
y_pred_cart_camel = decisionTreeModel.predict(X_test)

In [ ]:
# now, let's evaluate the code
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print(f'Accuracy: {accuracy_score(y_test, y_pred_cart_camel):.2f}')
print(f'Precision: {precision_score(y_test, y_pred_cart_camel, average="weighted"):.2f}, Recall: {recall_score(y_test, y_pred_cart_camel, average="weighted"):.2f}')

In [ ]:
from sklearn.tree import export_text

tree_rules = export_text(decisionTreeModel, feature_names=list(X_train.columns))

print(tree_rules)

# Training a more opaque classifier - Random Forest

In this example we train a model that does not provide the possibility to look into the decision process - it is more opaque. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

randomForestModel = RandomForestClassifier()
randomForestModel.fit(X_train, y_train)
y_pred_rf = randomForestModel.predict(X_test)

In [ ]:
# now, let's evaluate the code
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print(f'Accuracy: {accuracy_score(y_test, y_pred_rf):.2f}')
print(f'Precision: {precision_score(y_test, y_pred_rf, average="weighted"):.2f}, Recall: {recall_score(y_test, y_pred_rf, average="weighted"):.2f}')

In [ ]:
# now, let's check which of the features are the most important ones
# first we create a dataframe from this list
# then we sort it descending
# and then filter the ones that are not imporatnt
dfImportantFeatures = pd.DataFrame(randomForestModel.feature_importances_, index=X.columns, columns=['importance'])

# sorting values according to their importance
dfImportantFeatures.sort_values(by=['importance'], 
                                ascending=False, 
                                inplace=True)

# choosing only the ones that are important, skipping
# the features which have importance of 0
dfOnlyImportant = dfImportantFeatures[dfImportantFeatures['importance'] != 0]

# print the results
print(f'All features: {dfImportantFeatures.shape[0]}, but only {dfOnlyImportant.shape[0]} are used in predictions. ')

In [ ]:
# we use matplotlib and seaborn to make the plot
import matplotlib.pyplot as plt
import seaborn as sns

# Define size of bar plot
# We make the x axis quite much larger than the y-axis since
# there is a lot of features to visualize
plt.figure(figsize=(40,10))

# plot Searborn bar chart
# we just use the blue color
sns.barplot(y=dfOnlyImportant['importance'], 
            x=dfOnlyImportant.index, 
            color='steelblue')

# we make the x-labels rotated so that we can fit
# all the features
plt.xticks(rotation=90)

sns.set(font_scale=6)

# add chart labels
plt.title('Importance of features, in descending order')
plt.xlabel('Feature importance')
plt.ylabel('Feature names')

# Neural networks

The demo below shows how we can use neural networks in PyTorch to achieve the same results - training on the Camel data.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network architecture
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Define the hyperparameters
input_size = X_train.shape[1]  # Number of input features
hidden_size = 64              # Number of neurons in the hidden layer
num_classes = 2               # Number of output classes

# Create an instance of the neural network
model = NeuralNetwork(input_size, hidden_size, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert the data to PyTorch tensors
X_train_tensor = torch.Tensor(X_train.values)
y_train_tensor = torch.LongTensor(y_train.values)
X_test_tensor = torch.Tensor(X_test.values)

# Training the neural network
num_epochs = 10000
batch_size = 32
for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss at the end of each epoch
    if (epoch % 100 == 0):
      print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.3f}")

# Testing the neural network
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    y_pred_nn = predicted.numpy()


In [ ]:
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    X_test_tensor = torch.Tensor(X_test.values)
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    y_pred_nn = predicted.numpy()

In [ ]:
# now, let's evaluate the code
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print(f'Accuracy: {accuracy_score(y_test, y_pred_nn):.2f}')
print(f'Precision: {precision_score(y_test, y_pred_nn, average="weighted"):.2f}, Recall: {recall_score(y_test, y_pred_nn, average="weighted"):.2f}')

# Data leak

The following code illustrates a mistake when we use parts of the same data to both train and test the model. This problem is called the data leak problem, as we have the same data leaking from training to validation.

In [ ]:
# first, we need to prepare the data that can leak
# for this, we use the split function with different parameters
# we use 20% of the data to test the model
# which means that at least 10% of the data is the same as in the training set
X_trainL, X_testL, y_trainL, y_testL = \
        sklearn.model_selection.train_test_split(X, y, random_state=42, train_size=0.8)

In [ ]:
# now, let's evaluate the model on this new data
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    X_test_tensor = torch.Tensor(X_testL.values)
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    y_pred_nn = predicted.numpy()

In [ ]:
# now, let's evaluate the code
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print(f'Accuracy: {accuracy_score(y_testL, y_pred_nn):.2f}')
print(f'Precision: {precision_score(y_testL, y_pred_nn, average="weighted"):.2f}, Recall: {recall_score(y_testL, y_pred_nn, average="weighted"):.2f}')